In [1]:
import pickle
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import json 
import ast
import csv

In [2]:
movies=pd.read_csv('DataSets/movies_metadata.csv')

/home/abuzar/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (10,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
movies.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [3]:
credits=pd.read_csv('DataSets/credits.csv')

In [4]:
credits=credits.drop(credits.columns[2],axis=1)

In [5]:
Data=movies.join(credits)

In [6]:
cols=['id','genres','original_language','original_title','overview','cast','crew']

In [7]:
Final_data=Data[cols]

In [8]:
Final_data.dropna()

,id,genres,original_language,original_title,overview,cast,crew
0,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",en,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",en,Jumanji,When siblings Judy and Peter discover an encha...,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,15602,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",en,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,31357,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...","[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,11862,"[{'id': 35, 'name': 'Comedy'}]",en,Father of the Bride Part II,Just when George Banks has recovered from his ...,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."
...,...,...,...,...,...,...,...
45457,439050,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",fa,رگ خواب,Rising and falling between a man and woman.,"[{'cast_id': 2, 'character': 'Man with whisker...","[{'credit_id': '52fe4684c3a36847f810345f', 'de..."
45458,111109,"[{'id': 18, 'name': 'Drama'}]",tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,[],"[{'credit_id': '52fe4a74c3a368484e1542e9', 'de..."
45459,67758,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",en,Betrayal,"When one of her hits goes wrong, a professiona...","[{'cast_id': 1, 'character': 'Roo (voice)', 'c...","[{'credit_id': '52fe46249251416c7506e969', 'de..."
45460,227506,[],en,Satana likuyushchiy,"In a small town live two brothers, one a minis...","[{'cast_id': 3, 'character': 'All the members ...","[{'credit_id': '52fe478dc3a36847f813bd6b', 'de..."


In [9]:
crew_list=Final_data['crew']
cast_list=Final_data['cast']
movie_list=Final_data['genres']

0        [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
1        [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
2        [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...
3        [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
4                           [{'id': 35, 'name': 'Comedy'}]
                               ...                        
45457    [{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...
45458                        [{'id': 18, 'name': 'Drama'}]
45459    [{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...
45460                                                   []
45461                                                   []
Name: genres, Length: 45462, dtype: object

In [10]:
def ConvertToDict(ColumnsList,Resultant,Size):
    for i in range (0,Size,1):
        new_list=[]
        dict1=[]
        found=False
        for j in range(0,len(ColumnsList[i]),1):
            if (ColumnsList[i][j]=='{'):
                found=True
            elif ColumnsList[i][j]=='}':
                if ColumnsList[i][j]:
                    new_list+=ColumnsList[i][j]
                found=False
                list1=''.join(new_list)
                dict1.append(ast.literal_eval(list1))
                new_list=[]

            if found==True:
                if ColumnsList[i][j]:
                    new_list+=ColumnsList[i][j]
        Resultant.append(dict1)
    return Resultant


In [12]:
FinalCrewList=[]
FinalCrewList=ConvertToDict(crew_list,FinalCrewList,len(crew_list))

In [1]:
FinalMovieList=[]
FinalMovieList=ConvertToDict(movie_list,FinalMovieList,len(movie_list))

NameError: name 'ConvertToDict' is not defined

In [14]:
FinalCastList=[]
FinalCastList=ConvertToDict(cast_list,FinalCastList,len(movie_list))

In [34]:
crewlist = open('crewlist.pkl', 'wb')
pickle.dump(FinalCrewList, crewlist)

In [33]:
castlist = open('castlist.pkl', 'wb')
pickle.dump(FinalCastList, castlist)

In [35]:
movielist = open('movielist.pkl', 'wb')
pickle.dump(FinalMovieList, movielist)